In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
# for ensemble method use
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler



dummy_reg = DummyRegressor(strategy="mean")

# height removed
Test_Target = ["SUSPECT_ARRESTED_FLAG", "STOP_WAS_INITIATED",
                 "OBSERVED_DURATION_MINUTES", "SUSPECTED_CRIME_DESCRIPTION",
                 "STOP_DURATION_MINUTES", "FIREARM_FLAG",
                 "SUSPECT_RACE_DESCRIPTION",
                 "DEMEANOR_OF_PERSON_STOPPED", "SUSPECT_BODY_BUILD_TYPE","FRISKED_FLAG","SEARCHED_FLAG","STOP_LOCATION_BORO_NAME"]
Test1 = pd.read_excel("sqf2022.xlsx", usecols=Test_Target)

#print('MAX1:', Test1["OBSERVED_DURATION_MINUTES"].max())

#MS = Test1["STOP_DURATION_MINUTES"].values.reshape(-1, 1)
#scaler2 = StandardScaler()
#z_scores2 = scaler2.fit_transform(MS)

#outliers2 = np.where(z_scores2 > 1.5)
#Test1 = Test1.drop(outliers2[0], axis=0)
#print('MAX2:', Test1["STOP_DURATION_MINUTES"].max())



Feature_test=["FIREARM_FLAG","FRISKED_FLAG","SEARCHED_FLAG"]
#C_Feature_test=["STOP_WAS_INITIATED","SUSPECTED_CRIME_DESCRIPTION","SUSPECT_RACE_DESCRIPTION","DEMEANOR_OF_PERSON_STOPPED"]
value_counts = Test1['SUSPECT_ARRESTED_FLAG'].value_counts()
print(value_counts)




Test1['FIREARM_FLAG'] = Test1['FIREARM_FLAG'].replace('(null)', 'No')
Test1.replace('(null)', np.nan, inplace=True)
print(Test1.isna().sum())

lb = LabelBinarizer()
Test1["SUSPECT_ARRESTED_FLAG"] = lb.fit_transform(Test1["SUSPECT_ARRESTED_FLAG"])
Test1["FIREARM_FLAG"] = lb.fit_transform(Test1["FIREARM_FLAG"])
Test1["FRISKED_FLAG"] = lb.fit_transform(Test1["FRISKED_FLAG"])
Test1["SEARCHED_FLAG"] = lb.fit_transform(Test1["SEARCHED_FLAG"])


Test1["SUSPECT_ARRESTED_FLAG"].fillna(Test1["SUSPECT_ARRESTED_FLAG"].median,inplace=True)
Test1["FIREARM_FLAG"].fillna(Test1["FIREARM_FLAG"].median,inplace=True)
Test1["FRISKED_FLAG"].fillna(Test1["FRISKED_FLAG"].median,inplace=True)
Test1["SEARCHED_FLAG"].fillna(Test1["SEARCHED_FLAG"].median,inplace=True)

mode_val = Test1["SUSPECT_RACE_DESCRIPTION"].mode()[0]
Test1["SUSPECT_RACE_DESCRIPTION"].fillna(mode_val, inplace=True)



Test1.to_csv('output.csv', index=False)
#one Hot encoding
OneHot_Encode_feature = ["STOP_WAS_INITIATED","SUSPECTED_CRIME_DESCRIPTION"]
Label_En = OneHotEncoder(handle_unknown="ignore")
Feature_Encoded = Label_En.fit_transform(Test1[OneHot_Encode_feature])
Label_df = pd.DataFrame(Feature_Encoded.toarray(), columns=Label_En.get_feature_names_out(OneHot_Encode_feature))
#Label_df.fillna(value=0, inplace=True)
#Label_df.fillna(median)


#label encoding
LE=LabelEncoder()
Test1['SUSPECT_BODY_BUILD_TYPE']=LE.fit_transform(Test1["SUSPECT_BODY_BUILD_TYPE"])
Test1['SUSPECT_BODY_BUILD_TYPE'].fillna(Test1['SUSPECT_BODY_BUILD_TYPE'].mean, inplace=True)
#test is there still nan
print("new",Test1.isna().sum())

#cat_feature=list(Label_df.columns)+['SUSPECT_BODY_BUILD_TYPE']
#with everything, un comment the below code to include both Categorical data and numeric in RandomForest
#cat_feature=list(Label_df.columns)+['SUSPECT_BODY_BUILD_TYPE']+Feature_test
cat_feature=list(Label_df.columns)+Feature_test


Test1 = pd.concat([Test1, Label_df], axis=1)

# drop the original categorical feature column
Test1 = Test1.drop(columns=OneHot_Encode_feature)

#cols = ['OBSERVED_DURATION_MINUTES', 'STOP_DURATION_MINUTES']

# Create a StandardScaler object
#scaler = StandardScaler()

# Standardize the features
#Test1[cols] = scaler.fit_transform(Test1[cols])

# Define the threshold for outlier detection
#threshold = 3

# Calculate the absolute z-score for each data point
#z = np.abs(Test1[cols].values)

# Identify the rows that contain outliers
#outliers = np.where(z > threshold)[0]

# Remove the rows that contain outliers
#Test1 = Test1.drop(outliers, axis=0)

# Inverse transform the standardized data to obtain the original values
#Test1[cols] = scaler.inverse_transform(Test1[cols])
#print(Test1['OBSERVED_DURATION_MINUTES'].max())
#print(Test1['STOP_DURATION_MINUTES'].max())

train_data,test_data=train_test_split(Test1,test_size=0.2,random_state=42)

# TODO: add linear regression to models: LinearRegression()
models = [RandomForestClassifier(random_state=42),RandomForestClassifier(random_state=42),
            LogisticRegression(random_state=0,max_iter=1000),DecisionTreeClassifier(random_state=42),MultinomialNB()
         ]



# extract the categorical features and target variable from the training and testing data
#Random Forest
X_train = train_data[cat_feature] # training features_for Random forest
y_train = train_data['SUSPECT_ARRESTED_FLAG'] # training target variable for random forest

X_test = test_data[cat_feature] # testing features
y_test = test_data['SUSPECT_ARRESTED_FLAG'] # testing target variable

#Linear regression
X_train_linear=train_data[Feature_test] 
y_train_linear=train_data["SUSPECT_ARRESTED_FLAG"]

X_test_linear=test_data[Feature_test]
y_test_linear=test_data['SUSPECT_ARRESTED_FLAG']

#count Nan in dataset
#print(Test1.isna().sum())
# LogisticRegression(solver='lbfgs', max_iter=1000)
# models 
Test1.to_csv('output.csv')



N    10117
Y     4985
Name: SUSPECT_ARRESTED_FLAG, dtype: int64
STOP_WAS_INITIATED                0
OBSERVED_DURATION_MINUTES         0
SUSPECTED_CRIME_DESCRIPTION       0
STOP_DURATION_MINUTES             0
SUSPECT_ARRESTED_FLAG             0
FRISKED_FLAG                      0
SEARCHED_FLAG                     0
FIREARM_FLAG                      0
DEMEANOR_OF_PERSON_STOPPED     1907
SUSPECT_RACE_DESCRIPTION        192
SUSPECT_BODY_BUILD_TYPE         573
STOP_LOCATION_BORO_NAME           0
dtype: int64
new STOP_WAS_INITIATED                0
OBSERVED_DURATION_MINUTES         0
SUSPECTED_CRIME_DESCRIPTION       0
STOP_DURATION_MINUTES             0
SUSPECT_ARRESTED_FLAG             0
FRISKED_FLAG                      0
SEARCHED_FLAG                     0
FIREARM_FLAG                      0
DEMEANOR_OF_PERSON_STOPPED     1907
SUSPECT_RACE_DESCRIPTION          0
SUSPECT_BODY_BUILD_TYPE           0
STOP_LOCATION_BORO_NAME           0
dtype: int64


In [227]:
print(np.any(np.isnan(X_train)))
print(np.any(np.isinf(X_train)))
print(np.any(np.isnan(X_train)))
print(np.any(np.isinf(X_train)))
print("new",Test1.isna().sum())

False
False
False
False
new OBSERVED_DURATION_MINUTES                                                     0
STOP_DURATION_MINUTES                                                         0
SUSPECT_ARRESTED_FLAG                                                         0
FRISKED_FLAG                                                                  0
SEARCHED_FLAG                                                                 0
FIREARM_FLAG                                                                  0
DEMEANOR_OF_PERSON_STOPPED                                                 1907
SUSPECT_RACE_DESCRIPTION                                                      0
SUSPECT_BODY_BUILD_TYPE                                                       0
STOP_LOCATION_BORO_NAME                                                       0
STOP_WAS_INITIATED_Based on C/W on Scene                                      0
STOP_WAS_INITIATED_Based on Radio Run                                         0
STOP_WAS_INI

In [228]:
Model_N=MultinomialNB()
Model_N.fit(X_train,y_train)
N_prediction=Model_N.predict(X_test)
train_N_prediction=Model_N.predict(X_train)
N_train_accuracy=accuracy_score(y_train,train_N_prediction)
N_accuracy=accuracy_score(y_test,N_prediction)

print('Train_Accuracy:', N_accuracy)
print('Accuracy:', N_train_accuracy)



Train_Accuracy: 0.8391261171797418
Accuracy: 0.8370168032447645


In [229]:



model_R = RandomForestClassifier(random_state=42)
#model_R.fit(train_data[Feature_test], train_data["SUSPECT_ARRESTED_FLAG"])

#with only caterogical data but transform to numeric/ with both

model_R.fit(X_train,y_train)
Prediction=model_R.predict(X_test)

train_Prediction =model_R.predict(X_train)


train_accuracy=accuracy_score(y_train,train_Prediction)

accuracy = accuracy_score(y_test, Prediction)


print('Train_Accuracy:', train_accuracy)
print('Accuracy:', accuracy)


Train_Accuracy: 0.8673950831884778
Accuracy: 0.8679245283018868


In [230]:
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions on the testing and training data, and calculate RMSE
y_test_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)

print('Testing RMSE:', test_rmse)
print('Training RMSE:', train_rmse)

Testing RMSE: 0.34058471641017063
Training RMSE: 0.345491521615495


In [231]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
log_predict=logreg.predict(X_test)
log_score=accuracy_score(y_test,log_predict)

train_log_predict=logreg.predict(X_train)
train_log_score=accuracy_score(y_train,train_log_predict)

print("Accuracy:",log_score)
print("Train Accuract:",train_log_score)

Accuracy: 0.8659384309831182
Train Accuract: 0.8611042132273818


In [232]:
Dec_model=DecisionTreeClassifier(random_state=42)
Dec_model.fit(X_train,y_train)
Dec_predict=Dec_model.predict(X_test)
Train_Dec_predict=Dec_model.predict(X_train)
Dec_accuracy=accuracy_score(y_test,Dec_predict)
Train_Dec_accuracy=accuracy_score(y_train,Train_Dec_predict)
print("Decision Tree score: ",Dec_accuracy)
print("Train_Decision Tree score: ",Train_Dec_accuracy)


Decision Tree score:  0.8652763985435287
Train_Decision Tree score:  0.8673950831884778


In [233]:
dummy_reg=DummyRegressor(strategy="mean")
dummy_reg.fit(X_train,y_train)
y_train_predict=dummy_reg.predict(X_train)

trainerror=mean_squared_error(y_train,y_train_predict,squared=False)


y_pred=dummy_reg.predict(X_test)
testerror=mean_squared_error(y_test,y_pred,squared=False)

print("Training error",trainerror)
print("TestError:", testerror)







Training error 0.47084552628064064
TestError: 0.46784965294918474


In [234]:
for model in models:
    print(model, '\n')
    score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()
    print('Cross-Validation Accuracy:', score, '\n', '\n')

estimators=[("logreg", logreg), ("rf", model_R), ("Dc", Dec_model),("Linear",model),("Naive",Model_N)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train, y_train)
#test our model on the test data
print(ensemble.score(X_test, y_test))

RandomForestClassifier(random_state=42) 

Cross-Validation Accuracy: 0.8620978746023613 
 

RandomForestClassifier(random_state=42) 

Cross-Validation Accuracy: 0.8620978746023613 
 

LogisticRegression(max_iter=1000, random_state=0) 

Cross-Validation Accuracy: 0.8614354859480446 
 

DecisionTreeClassifier(random_state=42) 

Cross-Validation Accuracy: 0.860938762956651 
 

MultinomialNB() 

Cross-Validation Accuracy: 0.8336231426402934 
 

0.8689175769612711


In [144]:
nlpp = pd.read_excel("sqf2022.xlsx", usecols=["DEMEANOR_OF_PERSON_STOPPED"])
nlp = nlpp.fillna(value='neutral')


In [145]:
# Assuming df is your DataFrame and 'column_name' is the name of the column you want to check
null_count = nlp['DEMEANOR_OF_PERSON_STOPPED'].isnull().sum()
print(len(nlp))
print(f"Number of null values in 'column_name': {null_count}")

15102
Number of null values in 'column_name': 0


In [ ]:
dataa = nlp['DEMEANOR_OF_PERSON_STOPPED'].tolist()
dataa

In [156]:
analyzer = SentimentIntensityAnalyzer()
new_data = [ analyzer.polarity_scores(element)['compound'] for element in dataa]

In [160]:
data = zip(dataa, new_data)

In [163]:
new=[]
for pair in data:
    new.append(pair)


In [168]:
# Create DataFrame from list of tuples
final_df = pd.DataFrame(new, columns=['DEMEANOR_OF_PERSON_STOPPED', 'COMPOUND_SENTIMENT'])

In [166]:
final_df

,DEMEANOR_OF_PERSON_STOPPED,COMPOUND_SENTIMENT
0,FLED ON FOOT,0.0000
1,neutral,0.0000
2,NERVOUS CHANGING DIRECTION FROM OFFICERS RUNNING,-0.2732
3,CALM,0.3182
4,CALM,0.3182
...,...,...
15097,neutral,0.0000
15098,CALM,0.3182
15099,COMPLIANT,0.0000
15100,ANNOYED,-0.3818
